<a href="https://colab.research.google.com/github/B-Lorentz/physics-bsc-visual/blob/main/15_Quantum_Statistics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from matplotlib import pyplot as plt
from jax import numpy as jnp